<a href="https://colab.research.google.com/github/victorist/jira_python_api/blob/main/AzurAir_SMS_messages_import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Модуль импорта сообщений из таблиц БД Барьер-1

In [2]:
# Подключение к Google-диску, с дата-сетом.
# При запуске модуля будет показана ссылка - кликайте на неё и входите через свой аккаунт.
# Скопируйте ключ и вставьте его в появивщееся окошко.
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
# path for data save
dir = '/gdrive/Shareddrives/Dev/AZUR_AIR/_Jira API/'
dirfrom = '/gdrive/Shareddrives/Dev/AZUR_AIR/Барьер-1 (AZUR-SMS)/Таблицы_Барьер-1/'

In [ ]:
#@title Install Jira package and imports
!pip install atlassian-python-api
print('\n atlassian-python-api installed')

from atlassian import Jira
from atlassian import ServiceDesk
import pandas as pd
print('\n Atlassian modules imported')

In [9]:
# Read login and token from csv-file
# This file has 2 columns: "login" and "token". The second line  contains the login and token values.
dirtoken = '/gdrive/Shareddrives/Dev/AZUR_AIR/_Jira API/token/'
filetoken = pd.read_csv(dirtoken + 'az_iva_token.csv')
login = filetoken.login[0]
token = filetoken.token[0]


# Read from Google Sheets

In [60]:
# Read Excel with data
sms_data = pd.read_excel(dirfrom + 'SF_MEMORANDUM_s(ready).xlsx', sheet_name=0, header=0, index_col=None)

FIELDS_SELECTED = [
                   'DateTakeOFFReal',
                   'FlightName',
                   'PlnTYP',
                   'PlnBort',
                   'MEMORANDUM',
                   'AirPortTOKOD',
                   'AirPortLAKOD',
                   'KVSName',
                   'AdressMsg',
                   'MestoSob',
                   'FazaSob',
                   'KeyWord',
                   'ID_ISPOLN',
                   'ID_INSPECTOR'
                  ]
sms_data = sms_data[FIELDS_SELECTED]
sms_data = sms_data.sort_values('DateTakeOFFReal')

print('\nInformation about dataset\n', sms_data.info())
sms_data.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1430 entries, 1429 to 0
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DateTakeOFFReal  1430 non-null   datetime64[ns]
 1   FlightName       1430 non-null   object        
 2   PlnTYP           1430 non-null   object        
 3   PlnBort          1430 non-null   object        
 4   MEMORANDUM       1429 non-null   object        
 5   AirPortTOKOD     1430 non-null   object        
 6   AirPortLAKOD     1430 non-null   object        
 7   KVSName          1420 non-null   object        
 8   AdressMsg        1430 non-null   object        
 9   MestoSob         1417 non-null   object        
 10  FazaSob          1419 non-null   object        
 11  KeyWord          1418 non-null   object        
 12  ID_ISPOLN        1430 non-null   object        
 13  ID_INSPECTOR     1430 non-null   object        
dtypes: datetime64[ns](1), object(13)
memory 

,DateTakeOFFReal,FlightName,PlnTYP,PlnBort,MEMORANDUM,AirPortTOKOD,AirPortLAKOD,KVSName,AdressMsg,MestoSob,FazaSob,KeyWord,ID_ISPOLN,ID_INSPECTOR
1429,2014-12-18 19:20:00,ZF - 3992,B-757-200,VQ-BEZ,Задержка в связи с ожиданием разрешения \nавиа...,AYT,HRG,NaN,.,NaN,NaN,NaN,-,-
1428,2014-12-22 05:32:00,ZF - 3089,B-757-200,VQ-BEZ,Противообледенительня обработка ВС DL75,DME,HRG,NaN,.,NaN,NaN,NaN,-,-
1427,2014-12-22 11:41:00,ZF - 3090,B-757-200,VQ-BEZ,ППС KTK3089 + устранение неисправности БКО + о...,HRG,DME,NaN,.,NaN,NaN,NaN,-,-
1426,2014-12-23 00:55:00,ZF - 3091,B-757-200,VQ-BEZ,задержка в связи отсутствием договора на прием...,DME,SSH,NaN,.,NaN,NaN,NaN,-,-
1425,2014-12-23 12:45:00,ZF - 3335,B-757-200,VQ-BEZ,Отсутствие трансфера для экипажа при замене сб...,ROV,SSH,NaN,.,NaN,NaN,NaN,-,-


## Data for the period of interest

In [61]:
import datetime as dt

today = dt.datetime.today().strftime('%d-%m-%Y')  

# select date period
start_date = '01-01-2021'
end_date = today
print('Selected period from', start_date,'to',end_date)

# creat date mask
mask = (sms_data['DateTakeOFFReal'] > start_date) & (sms_data['DateTakeOFFReal'] <= end_date)

# assign mask to sms_data to return the rows with DateTakeOFFReal between our specified start/end dates
sms_data = sms_data.loc[mask]
sms_data

Selected period from 01-01-2021 to 28-10-2021


,DateTakeOFFReal,FlightName,PlnTYP,PlnBort,MEMORANDUM,AirPortTOKOD,AirPortLAKOD,KVSName,AdressMsg,MestoSob,FazaSob,KeyWord,ID_ISPOLN,ID_INSPECTOR
67,2021-01-09 13:32:00,377,B-757-200,VP-BLV,ДС от 09.01.2021 13:32:00\n\n09.01.2021 Рейс Z...,PEE,AER,Блинков Э.Е.,ИБП,ПЕРМЬ,ПОДГОТОВКА к ВЫЛЕТУ,Нарушение перевозки буфетно-кухонного оборудов...,-,-
66,2021-01-10 00:48:00,6681,B-737-800,VP-BYD,ДС от 10.01.2021 0:48:00\n\n10.01.21г. выполне...,SVX,VKO,Тирский А.А.,ИБП,МОСКВА (ВНК),ЗАХОД на ПОСАДКУ,"НСВП. Риски, связанные с инструментальными зах...",-,-
65,2021-01-10 09:59:00,346,B-757-200,VP-BPB,ДС от 10.01.2021 9:59:00\n\n\nЯ Белинский Миха...,AER,KZN,Белинский. М.Е.,ИБП,КАЗАНЬ,ЗАХОД на ПОСАДКУ,Установка давления на эшелоне перехода.,-,-
64,2021-01-11 22:17:00,2026,B-767-300,VP-BRA,ДС от 11.01.2021 22:17:00\n\nТепловизор на ВС ...,KJA,CDG,Севостьянов В.И.,ИБП,КРАСНОЯРСК,ПОЛЕТ,Тепловизор на ВС отсутствует.,-,-
63,2021-01-12 14:05:00,8882,B-757-200,VQ-BKB,ДС от 12.01.2021 14:05:00\n\n-Lavatory C: stro...,AYT,VKO,Ханзаров И.З.,ИБП,АНТАЛЬЯ,ПОЛЕТ,Отчет по рейсу.,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2021-06-02 17:57:00,344,B-737-800,VQ-BAL,ДС от 02.06.2021 17:57:00\n\n02.06.21 AZV 344 ...,AER,LED,Чистяков С.А.,ИБП,СОЧИ,НАБОР ВЫСОТЫ,Overspeed.,-,-
3,2021-06-02 20:41:00,6673,B-767-300,VQ-BEO,"ДС от 02.06.2021 20:41:00\n\nКоллеги,добрый де...",SVX,BHK,Снегирев А.И.,ИБП,БУХАРА,ЗАХОД на ПОСАДКУ,Ошибочное нажатие GA switch .,-,-
2,2021-06-04 05:00:00,7807,B-767-300,VP-BUX,ДС_1 от 04.06.2021 5:00:00\n\nЗдравствуйте! Не...,LED,NBE,Банев В.Н.,ИБП,С-ПЕТЕРБУРГ (Пулково),ПОДГОТОВКА к ВЫЛЕТУ,Не предоставляется такси экипажу при раннем вы...,-,-
1,2021-06-04 11:25:00,8405,B-737-900,VQ-BYO,ДС от 04.06.2021 11:25:00\n\n\n04.06.21 ULLI A...,LED,AER,Чистяков С.А.,ИБП,С-ПЕТЕРБУРГ (Пулково),ПОДГОТОВКА к ВЫЛЕТУ,Перронная проверка.,-,-


In [62]:
sms_data['PlnTYP'].unique()

array(['B-757-200', 'B-737-800', 'B-767-300', 'B-777-300', 'B-737-900'],
      dtype=object)

In [63]:
sms_data['PlnBort'].unique()

array(['VP-BLV', 'VP-BYD', 'VP-BPB', 'VP-BRA', 'VQ-BKB', 'VP-BXW',
       'VP-BUX', 'VP-BUY', 'VP-BUV', 'VQ-BEN', 'VQ-BZA', 'VQ-BZY',
       'VQ-BEO', 'RA-73032', 'VP-BIR', 'VQ-BAL', 'VQ-BZD', 'VQ-BUO',
       'VQ-BZF', 'RA-73034', 'VQ-BKF', 'VQ-BYX', 'RA-73030', 'VQ-BZC',
       'VQ-BQA', 'VP-BYC', 'VQ-BYO'], dtype=object)

# Explore fiealds


In [33]:
#@title Install Jira package and imports
%%capture
!pip install jira;
from jira import JIRA

In [34]:
# авторизация по примеру из https://habr.com/ru/post/442078/
jira_options = {'server': 'https://azurair.atlassian.net/'}

# используем вместо пароля токен - azur_jira
jira_az = JIRA(options=jira_options, basic_auth=(login, token))

In [58]:
# > список проектов
prj_list = jira_az.projects()
prj_list

[<JIRA Project: key='CAB', name='CAB', id='10002'>,
 <JIRA Project: key='MNT', name='MNT', id='10003'>,
 <JIRA Project: key='SMSG', name='Группа СУБП', id='10004'>,
 <JIRA Project: key='FSMS', name='СУБП', id='10000'>]

In [59]:
# > получаем список задач в проекте
jql = 'project = ' + 'FSMS'
issues_list = jira_az.search_issues(jql)
print(issues_list)

[<JIRA Issue: key='FSMS-455', id='10535'>, <JIRA Issue: key='FSMS-454', id='10534'>, <JIRA Issue: key='FSMS-453', id='10533'>, <JIRA Issue: key='FSMS-452', id='10532'>, <JIRA Issue: key='FSMS-451', id='10531'>, <JIRA Issue: key='FSMS-450', id='10530'>, <JIRA Issue: key='FSMS-449', id='10529'>, <JIRA Issue: key='FSMS-448', id='10528'>, <JIRA Issue: key='FSMS-447', id='10527'>, <JIRA Issue: key='FSMS-446', id='10526'>, <JIRA Issue: key='FSMS-445', id='10525'>, <JIRA Issue: key='FSMS-444', id='10524'>, <JIRA Issue: key='FSMS-443', id='10523'>, <JIRA Issue: key='FSMS-442', id='10522'>, <JIRA Issue: key='FSMS-441', id='10521'>, <JIRA Issue: key='FSMS-440', id='10520'>, <JIRA Issue: key='FSMS-439', id='10519'>, <JIRA Issue: key='FSMS-438', id='10518'>, <JIRA Issue: key='FSMS-437', id='10517'>, <JIRA Issue: key='FSMS-436', id='10516'>, <JIRA Issue: key='FSMS-435', id='10515'>, <JIRA Issue: key='FSMS-434', id='10514'>, <JIRA Issue: key='FSMS-433', id='10513'>, <JIRA Issue: key='FSMS-432', id='

In [ ]:
# > get the issue metadata
issue = 'FSMS-423'
meta = jira_az.editmeta(issue)
# meta
# meta_pd = pd.json_normalize(meta)
# meta_pd

# Jira Service Management
Python ServiceDesk module using
## Login to



In [55]:
# Autorization wih login and token
sd = ServiceDesk(
    url='https://azurair.atlassian.net',
    username = login,
    password = token)

## Creat issues from dataframe

In [56]:
print('Quantity raw in dataset:', len(sms_data.index))


Quantity raw in dataset: 68


In [64]:
# Create customer request
raw = 0

for raw in range(len(sms_data.index)):

  # loop body
  if sms_data.AdressMsg[raw] == 'СРВ':              # in the messages there are no problems dangerous for flights
    flight_safety = 'Не влияет на безопасность полёта'
  elif sms_data.AdressMsg[raw] == 'Врач':
    flight_safety = 'Влияет на безопасность полёта'       # about the lack of drugs
  else: flight_safety = 'Влияет на безопасность полёта'


  sd.create_customer_request(service_desk_id = 2,   # Service Desk ID
                              request_type_id = 30, # Request Type - 'email'
                              values_dict =
                              {
                              # 'issuetype': {'name': 'Проблема'},
                              'summary' : 'b~ '+ sms_data.KeyWord[raw],         # b1 - lable Барьер-1
                              'description': sms_data.MEMORANDUM[raw],
                              'customfield_10071' : sms_data.DateTakeOFFReal[raw].strftime('%Y-%m-%d %H:%M:%S'), # Event date (DateTakeOFFReal )
                              # 'customfield_10072' :{'value': 'ARH'},                   # Airport
                              'customfield_10073' : {'value': sms_data.PlnTYP[raw]},      # Aircraft type (PlnTYP)
                              'customfield_10058' : {'value': sms_data.PlnBort[raw]},      # Board number	(PlnBort)
                              'customfield_10059' : sms_data.FlightName[raw],                # Flight Number (FlightName)
                              'customfield_10056' : {'value' : flight_safety}            # flight_safety = yes/no

                              }   
                            )
  raw +=1


Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...
Creating request...


In [65]:
# Get customer requests and creat dataframe
customer_requests = sd.get_my_customer_requests()
df_data = pd.json_normalize(customer_requests)
df_data

,_expands,issueId,issueKey,requestTypeId,serviceDeskId,requestFieldValues,createdDate.iso8601,createdDate.jira,createdDate.friendly,createdDate.epochMillis,reporter.accountId,reporter.emailAddress,reporter.displayName,reporter.active,reporter.timeZone,reporter._links.jiraRest,reporter._links.avatarUrls.48x48,reporter._links.avatarUrls.24x24,reporter._links.avatarUrls.16x16,reporter._links.avatarUrls.32x32,reporter._links.self,currentStatus.status,currentStatus.statusCategory,currentStatus.statusDate.iso8601,currentStatus.statusDate.jira,currentStatus.statusDate.friendly,currentStatus.statusDate.epochMillis,_links.jiraRest,_links.web,_links.self,_links.agent
0,"[participant, status, sla, requestType, servic...",10603,FSMS-523,30,2,"[{'fieldId': 'customfield_10056', 'label': 'Вл...",2021-10-28T22:46:53+0300,2021-10-28T22:46:53.700+0300,Сегодня 10:46,1635450413700,61014170627b560068b945dc,victor.istratov@azurair.ru,Виктор Истратов,True,Etc/GMT-3,https://azurair.atlassian.net/rest/api/2/user?...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://azurair.atlassian.net/rest/api/2/user?...,Открыта,NEW,2021-10-28T22:46:53+0300,2021-10-28T22:46:53.700+0300,Сегодня 10:46,1635450413700,https://azurair.atlassian.net/rest/api/2/issue...,https://azurair.atlassian.net/servicedesk/cust...,https://azurair.atlassian.net/rest/servicedesk...,https://azurair.atlassian.net/browse/FSMS-523
1,"[participant, status, sla, requestType, servic...",10602,FSMS-522,30,2,"[{'fieldId': 'customfield_10056', 'label': 'Вл...",2021-10-28T22:46:52+0300,2021-10-28T22:46:52.367+0300,Сегодня 10:46,1635450412367,61014170627b560068b945dc,victor.istratov@azurair.ru,Виктор Истратов,True,Etc/GMT-3,https://azurair.atlassian.net/rest/api/2/user?...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://azurair.atlassian.net/rest/api/2/user?...,Открыта,NEW,2021-10-28T22:46:52+0300,2021-10-28T22:46:52.367+0300,Сегодня 10:46,1635450412367,https://azurair.atlassian.net/rest/api/2/issue...,https://azurair.atlassian.net/servicedesk/cust...,https://azurair.atlassian.net/rest/servicedesk...,https://azurair.atlassian.net/browse/FSMS-522
2,"[participant, status, sla, requestType, servic...",10601,FSMS-521,30,2,"[{'fieldId': 'customfield_10056', 'label': 'Вл...",2021-10-28T22:46:51+0300,2021-10-28T22:46:51.426+0300,Сегодня 10:46,1635450411426,61014170627b560068b945dc,victor.istratov@azurair.ru,Виктор Истратов,True,Etc/GMT-3,https://azurair.atlassian.net/rest/api/2/user?...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://azurair.atlassian.net/rest/api/2/user?...,Открыта,NEW,2021-10-28T22:46:51+0300,2021-10-28T22:46:51.426+0300,Сегодня 10:46,1635450411426,https://azurair.atlassian.net/rest/api/2/issue...,https://azurair.atlassian.net/servicedesk/cust...,https://azurair.atlassian.net/rest/servicedesk...,https://azurair.atlassian.net/browse/FSMS-521
3,"[participant, status, sla, requestType, servic...",10600,FSMS-520,30,2,"[{'fieldId': 'customfield_10056', 'label': 'Вл...",2021-10-28T22:46:50+0300,2021-10-28T22:46:50.453+0300,Сегодня 10:46,1635450410453,61014170627b560068b945dc,victor.istratov@azurair.ru,Виктор Истратов,True,Etc/GMT-3,https://azurair.atlassian.net/rest/api/2/user?...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://avatar-management--avatars.us-west-2.p...,https://azurair.atlassian.net/rest/api/2/user?...,Открыта,NEW,2021-10-28T22:46:50+0300,2021-10-28T22:46:50.453+0300,Сегодня 10:46,1635450410453,https:

In [66]:
df_data.requestFieldValues[0]

[{'fieldId': 'customfield_10056',
  'label': 'Влияние на безопасность',
  'value': {'id': '10110',
   'self': 'https://azurair.atlassian.net/rest/api/2/customFieldOption/10110',
   'value': 'Влияет на безопасность полёта'}},
 {'fieldId': 'summary',
  'label': 'Тема',
  'value': 'b~ Нарушение перевозки буфетно-кухонного оборудования.'},
 {'fieldId': 'description',
  'label': 'Описание',
  'renderedValue': {'html': '<p>ДС от 09.01.2021 13:32:00</p>\n\n<p>09.01.2021 Рейс ZF-377 PEE-AER при приеме БКО было обнаружено нарушение перевозки.<br>\nНа ВС VP-BLV находилась телега 1/2 в несоответствующей для неё по размерам нише и закреплялась только внешними фиксаторами ,что является нарушением.</p>\n\n<p>автор: Тихонова У.Б.</p>'},
  'value': 'ДС от 09.01.2021 13:32:00\n\n09.01.2021 Рейс ZF-377 PEE-AER при приеме БКО было обнаружено нарушение перевозки.\nНа ВС VP-BLV находилась телега 1/2 в несоответствующей для неё по размерам нише и закреплялась только внешними фиксаторами ,что является наруше

## *Create customer request example*

___

In [ ]:
'''
raw =0
# for raw in sms_data.index:

# creat issue
jira.issue_create(
    fields={
        "project": {"key": "FSMS"},
        "issuetype": {"name": "Проблема"},    # ""fields.issuetype.name""
        # "customfield_10010" : {"id": "30"},
        "summary": sms_data.KeyWord[raw],
        "description": sms_data.MEMORANDUM[raw],
        "customfield_10071" : sms_data.DateTakeOFFReal[raw],
        "fields.customfield_10073" : sms_data.PlnTYP[raw], # 	Тип воздушного судна (PlnTYP)

    }
)
  
print(sms_data.DateTakeOFFReal[raw])
raw +=1

print('Колличество импортированных в Jira сообщений:',raw)
# sms_data.DateTakeOFFReal[raw]
'''